数据封装、继承和多态是面向对象的基本特性，但是还有很多高级特性需要学习

## 使用__slots__

我们定义好class，创建对应的实例之后，可以给该实例绑定任何的属性和方法，这是动态语言的灵活性。

In [3]:
class Student(object):
    pass

In [6]:
def set_age(self, age):
    self.age = age

from types import MethodType 

s = Student()
s.set_age = MethodType(set_age, s)
s.set_age(25)
print(s.age)

25


In [7]:
s2 = Student()
s2.set_age(10)

AttributeError: 'Student' object has no attribute 'set_age'

In [9]:
def set_score(self, score):
    self.score = score
    
Student.set_score = set_score

In [11]:
s.set_score(100)
s.score

100

In [14]:
s2.set_score(99)
s2.score

99

In [23]:
class Student(object):
    __slots__ = ('name', 'age')

In [24]:
s = Student()
s.age = 25
s.score = 99

AttributeError: 'Student' object has no attribute 'score'

In [25]:
class GraduateStudent(Student):
    pass
g = GraduateStudent()
g.score = 9999

## 使用@property

绑定属性的时候，如果我们直接把属性暴露出去，虽然写起来简单，但是没办法修改检查参数，导致属性可以随便改。
这个时候可以用@property把方法变成属性
把一个getter方法变成属性，只需要加上@property就可以了，此时，@property本身会创造另一个装饰器@score.setter，负责把一个setter方法变成属性赋值，具体操作如下

In [1]:
class Student(object):

    @property
    def score(self):
        return self._score
    
    @score.setter
    def score(self, value):
        if not isinstance(value, int):
            raise ValueError('score must be an integer!')
        if value < 0 or value > 100:
            raise ValueError('score must be between 0~100')
        self._score = value

In [2]:
s = Student()
s.score = 60
s.score = 999

ValueError: score must be between 0~100

可以只定义getter方法，不定义setter方法，这样就是一个只读属性
另外需要注意属性的方法名和实例变量名不要重名

In [4]:
class Student(object):

    # 方法名称和实例变量均为birth:
    @property
    def birth(self):
        return self.birth ## 错误示范

#### 小结
@property 广泛应用在类的定义中，可以让调用者写出简短的代码，同时保证对参数进行必要的检查，这样程序运行可以减少出错的可能性。

## 多重继承

In [1]:
class Animal(object):
    pass

class Mammal(Animal): # 哺乳类
    pass

class Bird(Animal):  # 鸟类
    pass

class Dog(Mammal):
    pass

class Bat(Mammal):
    pass

class Parrot(Bird):
    pass

class Ostrich(Bird):
    pass

现在，我们要给动物再加上Runnale和Flyable的功能，只需要先定义好Runnable和Flyable的类：

In [4]:
class Runnable(object):
    def run(self):
        print("Running...")

class Flyable(object):
    def fly(self):
        print("Flying...")

对于需要Runnale功能的动物，就多继承Runnale，例如Dog

In [6]:
class Dog(Mammal, Runnable):
    pass

对于需要Flyable功能的动物，就多继承一个Flyable，例如Bat

In [7]:
class Bat(Mammal, Flyable):
    pass

通过多重继承，一个子类就可以同时获得多个父类的所有功能

#### Mixln

在设计类的继承关系时，通常，主线都是单一继承下来的，例如，Ostrich继承自Bird。但是，如果需要“混入”额外的功能，可以继承Runnable。这种设计通常称之为Mixin。

## 定制类

形如 \__xxx\__ 的变量或者函数名就要注意，这些在python当中有特殊用途的。

#### \__str\__

我们先定义一个student类，打印一个实例：

In [9]:
class Student(object):
    def __init__(self, name):
        self.name = name

In [10]:
print(Student('Michael'))

怎么打印出来的实例，不但好看，而且容易看出实例内部重要的数据。

但是细心的朋友会发现直接敲变量不用print，打印出来的实例还是不好看：

In [8]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)'%self.name
    
print(Student('Michael'))

Student object (name: Michael)


这样打印出来的实例，不但好看，而且容易看出实例内部重要的数据。

但是细心的朋友会发现直接敲变量不用print，打印出来的实例还是不好看：

这是因为直接显示变量调用的不是 \__str\__()，两者的区别是 \__str\__() 返回用户看到的字符串，而 \__repr\__()返回程序开发者看到的字符串，也就是说，\__repr\__()是为调试服务的。解决办法是再定义一个 \__repr\__()。但是通常\__str\__()和\__repr\__() 代码都是一样的，所以，有个偷懒的写法：

In [12]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__

#### \__iter\__

如果一个类想被用于for...in 循环，类似list或tuple那样，就要必须实现一个\__iter\__()方法，该方法返回一个迭代对象，然后，Python的for循环就会不调
用该迭代对象的\__next\__()方法拿到循环的下一个值，直到遇到stopIteration错误时退出循环。

我们以斐波那契数列为例，写一个Fib类，可以作用于for循环：

In [15]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.a, self.b = self.b, self.a + self.b
        if self.a > 100000:
            raise StopIteration()
        return self.a

现在，试试把Fib实例作用于for循环：

In [16]:
for n in Fib():
    print(n)

1
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025


#### __iter__

如果一个类想被用于for...in循环，类似list或tuple那样，就必须实现一个\__iter\__()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的\__next\__()